In [8]:
# !jupyter nbextension enable code_prettify/code_prettify

In [9]:
import os
#set proxy

os.environ['HTTP_PROXY'] = 'http://fp.cs.ovgu.de:3210/'
os.environ['HTTPS_PROXY'] = 'http://fp.cs.ovgu.de:3210/'

In [10]:
# pip install yapf

In [11]:
# import preprocess.py
# import cam.py
# import perturb.py
# import metrics.py

import datetime

In [12]:
# 1. generate and save ground truth saliency map
# 2. save a version over the image
def inference(model, saliency_method_fn):
    gpu_reference_tensor = next(model.parameters())
    tensor = tensor.type_as(gpu_reference_tensor)
    target_layer = model.layer4[1].conv2
    wrapped_model = saliency_method_fn(torch_model, target_layer)
    cam, idx = wrapped_model(tensor)
    fig, ax = plt.subplots()
    print(idx)
    img = reverse_normalize(tensor)
    heatmap = visualize(img, cam)
    heatmap = np.transpose(heatmap.squeeze(), (1, 2, 0))
    ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
    ax.set_title(f"Class: {classes[idx]}")
    fig.savefig(f"/project/validating_attribution_techniques/shardul/output/saliency/before_noise/{idx2label[idx]}.jpeg")
    

In [13]:
def load_model(model_architecture):
    use_cuda = True
    device = torch.device("cuda" if use_cuda else "cpu")
    model = torch.hub.load('pytorch/vision:v0.10.0',model_architecture, pretrained=True).to(device)
    model.eval()
    return model

In [14]:
def run_pipeline(op_dir, sal_method_list, vision_model, perturb_tech):
    current_time = datetime.datetime.now()
    if not os.path.exists(op_dir + current_time):
            os.mkdir(op_dir + current_time)
    torch_model = load_model(vision_model)
    for method in sal_method_list:
        inference(torch_model, method)
        printf('inference complete for'+ method)
    for technique in perturb_tech:
        occu_perturb()
        noise_perturb()
        # 1. generate and save perturbed image
        # 2. save saliency map of the perturbed image    
        #loop over the folder to compare the saliency maps of 2 images
        #location_based
        nss_score = nss(gt_saliency, perturbed_saliency)
        #distribution_based
        emd_score = emd(gt_saliency, perturbed_saliency)
        sim_score = sim(gt_saliency, perturbed_saliency)
        print(f'nss =', nss, 'emd =', emd)
    return

In [15]:
op_dir = 'commons/experiment/'
model_arch = 'resnet50'

